# PyTorch Tutorial

기초 PyTorch 문법들을 정리해 봅시다.

In [1]:
import torch
import numpy as np

PyTorch 버전을 프린트해 봅시다.

In [2]:
import sys
print(sys.version_info)
print(torch.__version__)

sys.version_info(major=3, minor=9, micro=10, releaselevel='final', serial=0)
1.13.0.dev20220812


PyTorch는 엔비디아 GPU를 사용해 CUDA 가속을 할 수 있습니다.

In [3]:
if torch.cuda.is_available():
    !nvidia-smi
else:
    print("No CUDA capable device found")

No CUDA capable device found


## Tensor Initialization

먼저 PyTorch의 기본 자료형인 Tensor는 다음과 같이 만들 수 있습니다.

In [4]:
# 리스트를 통해 텐서 만들기
a = torch.tensor([1, 2, 3])
print(a)

# numpy 배열을 통해 텐서 만들기
a = torch.tensor(np.array([1, 2, 3]))
print(a)

# 텐서를 통해 리스트 만들기
a = list(torch.tensor([1, 2, 3]))
print(a)

# 텐서를 통해 numpy 배열 만들기
a = torch.tensor([1, 2, 3]).numpy()
print(a)

tensor([1, 2, 3])
tensor([1, 2, 3])
[tensor(1), tensor(2), tensor(3)]
[1 2 3]


```np.zeros```나 ```np.ones```와 유사하게 사용할 수도 있습니다. NumPy에서 가능했던 변환들이 Tensor에서도 대부분 비슷하게 됩니다.

In [5]:
# 0으로 초기화된 텐서 만들기
a = torch.zeros(3, 3)
print(a)

# 1으로 초기화된 텐서 만들기
a = torch.ones(3, 3)
print(a)

# 정수형 텐서 만들기
a = torch.tensor([1, 2, 3], dtype=torch.int64)
print(a)

# 실수형 텐서 만들기
a = torch.tensor([1, 2, 3], dtype=torch.float32)
print(a)

# 텐서의 요소를 각각 출력하기
a = torch.tensor([1, 2, 3])
print(a[0])

# 텐서 내에서 요소를 수정하기
a[0] = 10
print(a)

# 텐서의 크기 확인하기
print(a.size())

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
tensor([1, 2, 3])
tensor([1., 2., 3.])
tensor(1)
tensor([10,  2,  3])
torch.Size([3])


## Module

In [6]:
import torch.nn as nn


In [7]:
# 모듈을 사용하여 텐서 만들기
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 1, 3)
        self.conv2 = nn.Conv2d(1, 1, 3)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        return x

In [8]:
# 모듈을 사용하여 텐서 만들기
net = Net()
print(net)

# 텐서를 사용하여 모듈을 사용하기
x = torch.randn(1, 1, 28, 28)
out = net(x)
print(out.size())


Net(
  (conv1): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
)
torch.Size([1, 1, 24, 24])


In [9]:
import torch.nn.functional as F


In [10]:
# 조금 더 복잡한 모듈 만들기
class Attention(nn.Module):
    def __init__(self, feature_dim, step_dim, bias=True, **kwargs):
        super(Attention, self).__init__(**kwargs)

        self.feature_dim = feature_dim
        self.step_dim = step_dim
        self.query = nn.Linear(feature_dim, feature_dim, bias)
        self.key = nn.Linear(feature_dim, feature_dim, bias)
        self.value = nn.Linear(feature_dim, feature_dim, bias)

    def forward(self, x):
        step = x.shape[-2]
        feature = x.shape[-1]

        query = self.query(x)
        key = self.key(x)
        value = self.value(x)

        query = query.view(step, -1, feature)
        key = key.view(step, -1, feature)
        value = value.view(step, -1, feature)

        attention = torch.bmm(query, key.transpose(1, 2))
        attention = F.softmax(attention, dim=-1)
        attention = torch.bmm(attention, value)
        attention = attention.view(step, -1, feature)

        return attention

In [11]:
# 조금 더 복잡한 모듈 사용하기
net = Attention(1, 1)
print(net)



Attention(
  (query): Linear(in_features=1, out_features=1, bias=True)
  (key): Linear(in_features=1, out_features=1, bias=True)
  (value): Linear(in_features=1, out_features=1, bias=True)
)


## GPU Acceleration

PyTorch는 GPU 가속을 지원합니다. GPU 가속을 하기 위해서는 가속할 연산에 사용할 텐서들을 VRAM으로 옮겨 연산해야 합니다. 

DEVICE를 설정합니다. DEVICE는 보통 'cpu', 'cuda' 2가지가 있습니다.

만약 여러 개의 GPU를 활용한다면 DEVICE 뒤에 인덱스가 붙습니다. 

In [12]:
# device 설정하기
DEVICE = torch.device(
    "cuda"
    if torch.cuda.is_available()
    else ("mps" if torch.backends.mps.is_available() else "cpu")
)
DEVICE


device(type='mps')

별 언급이 없으면 디폴트로는 CPU memory에 텐서가 저장됩니다. ```to``` 메소드를 통해 정의된 텐서의 위치를 옮겨줄 수 있습니다.

In [13]:
# 텐서를 설정한 device로 만들기
a = torch.tensor([1, 2, 3], device=DEVICE)
print(a)
print(a.device)

tensor([1, 2, 3], device='mps:0')
mps:0


In [14]:
# CPU에서 텐서 생성하기
a = torch.tensor([1, 2, 3])
print(a.device)
a = a.to("cpu")
print(a.device)

cpu
cpu


In [15]:
# GPU에서 텐서 생성하기
a = torch.tensor([1, 2, 3])
print(a.device)
a = a.to(DEVICE)
print(a.device)

cpu
mps:0


### GPU Performance 측정

In [16]:
import time

In [17]:
# CPU 시간 측정하기
a = torch.ones(1000, 1000)
b = torch.ones(1000, 100)
start = time.perf_counter()
for _ in range(10000):
    (a@b).mean().item()
print(time.perf_counter() - start)

1.9628162920000012


In [18]:
# GPU 시간 측정하기
a = torch.ones(1000, 1000, device=DEVICE)
b = torch.ones(1000, 100, device=DEVICE)
start = time.perf_counter()
for _ in range(10000):
    (a@b).mean().item()
print(time.perf_counter() - start)

12.180795041000001


In [19]:
# 모듈을 이용해서 텐서 생성하기
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(100, 100)
        self.fc2 = nn.Linear(100, 100)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [20]:
# CPU에서 텐서 생성하기
module = Net()
print(module)

x = torch.zeros(100, 100)
start = time.perf_counter()
for _ in range(10000):
    module(x).mean().item()
print(time.perf_counter() - start)

Net(
  (fc1): Linear(in_features=100, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
)
0.5003753329999974


In [21]:
# GPU 에서 텐서 생성하기
module = Net().to(DEVICE)
print(module)

x = torch.zeros(100, 100, device=DEVICE)
start = time.perf_counter()
for _ in range(10000):
    module(x).mean().item()
print(time.perf_counter() - start)

Net(
  (fc1): Linear(in_features=100, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
)
5.791816541999999


## Optimizer 설정